In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,cross_validate,KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import GaussianNB
import weather_and_time_functions as weather_time

pd.set_option('display.max_columns',None)
scaler = StandardScaler()

In [2]:
flight_df = pd.read_feather('data/v2_clean_flight')
flight_test_df = pd.read_feather('data/v1_clean_flight_test')
fuel_consumption_df = pd.read_feather('data/v1_clean_fuel_consumption')
passenger_df = pd.read_feather('data/v1_clean_passenger')
og_flight_df = pd.read_feather('data/flights')

In [3]:
delay_cols = og_flight_df[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']].fillna(0)

In [4]:
col_w_values = delay_cols[(delay_cols['carrier_delay'] != 0) | 
                        (delay_cols['weather_delay'] != 0) | 
                        (delay_cols['nas_delay'] != 0) | 
                        (delay_cols['security_delay'] != 0) | 
                        (delay_cols['late_aircraft_delay'] != 0)]

og_cols_w_delays = og_flight_df[(delay_cols['carrier_delay'] != 0) | 
                        (delay_cols['weather_delay'] != 0) | 
                        (delay_cols['nas_delay'] != 0) | 
                        (delay_cols['security_delay'] != 0) | 
                        (delay_cols['late_aircraft_delay'] != 0)]

In [5]:
def highest_delay(array):
    """ 
        Takes an array and transforms it to be a row of 0's except for the highest number in that array and makes it 1

        Parameters:
            array(pandas array)
        
        Returns:
            An array consisting of 0's except for the largest number in that array is turned into 1
    """
    new_array=[]
    for num in array.values:
        max_num = np.max(num)
        num[num<max_num] = 0 
        new_array.append(num/max_num)
    return new_array
        

In [6]:
new_array_values = highest_delay(col_w_values)

In [7]:
col_w_values[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']] = new_array_values

C:\Users\John\AppData\Local\Temp\ipykernel_17220\2967611458.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  col_w_values[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']] = new_array_values


In [8]:
og_cols_w_delays[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']] = new_array_values

C:\Users\John\AppData\Local\Temp\ipykernel_17220\2788351538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  og_cols_w_delays[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']] = new_array_values


In [9]:
dep_hour = weather_time.obtain_depart_hour(og_cols_w_delays)
arr_hour = weather_time.obtain_depart_hour(og_cols_w_delays)
og_cols_w_delays['dep_hour'] = dep_hour
og_cols_w_delays['arr_hour'] = arr_hour

C:\Users\John\AppData\Local\Temp\ipykernel_17220\1945060044.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  og_cols_w_delays['dep_hour'] = dep_hour
C:\Users\John\AppData\Local\Temp\ipykernel_17220\1945060044.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  og_cols_w_delays['arr_hour'] = arr_hour


In [10]:
og_cols_w_delays['month'] = pd.to_datetime(og_cols_w_delays['fl_date'],yearfirst=True).dt.month
og_cols_w_delays['day'] = pd.to_datetime(og_cols_w_delays['fl_date'],yearfirst=True).dt.day

C:\Users\John\AppData\Local\Temp\ipykernel_17220\3057312728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  og_cols_w_delays['month'] = pd.to_datetime(og_cols_w_delays['fl_date'],yearfirst=True).dt.month
C:\Users\John\AppData\Local\Temp\ipykernel_17220\3057312728.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  og_cols_w_delays['day'] = pd.to_datetime(og_cols_w_delays['fl_date'],yearfirst=True).dt.day


In [11]:
num_cols = ['crs_elapsed_time','distance']
rank_cols = ['day','month','dep_hour','arr_hour']
dum_cols = ['origin','dest','op_unique_carrier']
y_target = og_cols_w_delays[['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay']]


In [12]:
flight_model = og_cols_w_delays[['crs_elapsed_time','distance','day','month','dep_hour','arr_hour','origin','dest','op_unique_carrier']]
flight_model = pd.get_dummies(flight_model, columns=dum_cols)
flight_model

,crs_elapsed_time,distance,day,month,dep_hour,arr_hour,origin_ABE,origin_ABI,origin_ABQ,origin_ABY,origin_ACK,origin_ACT,origin_ACV,origin_ACY,origin_ADQ,origin_AEX,origin_AGS,origin_ALB,origin_ALO,origin_ALW,origin_AMA,origin_ANC,origin_APN,origin_ART,origin_ASE,origin_ATL,origin_ATW,origin_AUS,origin_AVL,origin_AVP,origin_AZA,origin_AZO,origin_BDL,origin_BFL,origin_BFM,origin_BGM,origin_BGR,origin_BHM,origin_BIL,origin_BIS,origin_BJI,origin_BLI,origin_BLV,origin_BMI,origin_BNA,origin_BOI,origin_BOS,origin_BPT,origin_BQK,origin_BQN,origin_BRD,origin_BRO,origin_BRW,origin_BTM,origin_BTR,origin_BTV,origin_BUF,origin_BUR,origin_BWI,origin_BZN,origin_CAE,origin_CAK,origin_CDC,origin_CGI,origin_CHA,origin_CHO,origin_CHS,origin_CID,origin_CIU,origin_CKB,origin_CLE,origin_CLL,origin_CLT,origin_CMH,origin_CMI,origin_CMX,origin_CNY,origin_COD,origin_COS,origin_COU,origin_CPR,origin_CRP,origin_CRW,origin_CSG,origin_CVG,origin_CWA,origin_DAB,origin_DAL,origin_DAY,origin_DBQ,origin_DCA,origin_DEN,origin_DFW,origin_DHN,origin_DIK,origin_DLG,origin_DLH,origin_DRO,origin_DSM,origin_DTW,origin_DVL,origin_EAR,origin_EAT,origin_EAU,origin_ECP,origin_EGE,origin_ELM,origin_ELP,origin_ERI,origin_ESC,origin_EUG,origin_EVV,origin_EWN,origin_EWR,origin_EYW,origin_FAI,origin_FAR,origin_FAT,origin_FAY,origin_FCA,origin_FLG,origin_FLL,origin_FLO,origin_FNT,origin_FSD,origin_FSM,origin_FWA,origin_GCC,origin_GEG,origin_GFK,origin_GGG,origin_GJT,origin_GNV,origin_GPT,origin_GRB,origin_GRI,origin_GRK,origin_GRR,origin_GSO,origin_GSP,origin_GTF,origin_GUC,origin_HDN,origin_HHH,origin_HLN,origin_HNL,origin_HOB,origin_HOU,origin_HPN,origin_HRL,origin_HSV,origin_HTS,origin_HVN,origin_HYA,origin_HYS,origin_IAD,origin_IAG,origin_IAH,origin_ICT,origin_IDA,origin_ILM,origin_IMT,origin_IND,origin_INL,origin_IPT,origin_ISN,origin_ISP,origin_ITH,origin_ITO,origin_JAC,origin_JAN,origin_JAX,origin_JFK,origin_JHM,origin_JLN,origin_JMS,origin_JNU,origin_KOA,origin_KTN,origin_LAN,origin_LAR,origin_LAS,origin_LAW,origin_LAX,origin_LBB,origin_LBE,origin_LBF,origin_LBL,origin_LCH,origin_LCK,origin_LEX,origin_LFT,origin_LGA,origin_LGB,origin_LIH,origin_LIT,origin_LNK,origin_LNY,origin_LRD,origin_LSE,origin_LWB,origin_LWS,origin_LYH,origin_MAF,origin_MBS,origin_MCI,origin_MCO,origin_MDT,origin_MDW,origin_MEM,origin_MFE,origin_MFR,origin_MGM,origin_MHK,origin_MHT,origin_MIA,origin_MKE,origin_MKG,origin_MKK,origin_MLB,origin_MLI,origin_MLU,origin_MMH,origin_MOB,origin_MOT,origin_MQT,origin_MRY,origin_MSN,origin_MSO,origin_MSP,origin_MSY,origin_MTJ,origin_MVY,origin_MYR,origin_OAJ,origin_OAK,origin_OGD,origin_OGG,origin_OGS,origin_OKC,origin_OMA,origin_ONT,origin_ORD,origin_ORF,origin_ORH,origin_PAE,origin_PAH,origin_PBG,origin_PBI,origin_PDX,origin_PGD,origin_PGV,origin_PHF,origin_PHL,origin_PHX,origin_PIA,origin_PIE,origin_PIR,origin_PIT,origin_PLN,origin_PNS,origin_PQI,origin_PRC,origin_PSC,origin_PSE,origin_PSG,origin_PSM,origin_PSP,origin_PUW,origin_PVD,origin_PWM,origin_RAP,origin_RDD,origin_RDM,origin_RDU,origin_RFD,origin_RHI,origin_RIC,origin_RKS,origin_RNO,origin_ROA,origin_ROC,origin_RST,origin_RSW,origin_SAF,origin_SAN,origin_SAT,origin_SAV,origin_SBA,origin_SBN,origin_SBP,origin_SBY,origin_SCC,origin_SCE,origin_SCK,origin_SDF,origin_SEA,origin_SFB,origin_SFO,origin_SGF,origin_SGU,origin_SHD,origin_SHV,origin_SJC,origin_SJT,origin_SJU,origin_SLC,origin_SLN,origin_SMF,origin_SNA,origin_SPI,origin_SPS,origin_SRQ,origin_STC,origin_STL,origin_STS,origin_STT,origin_STX,origin_SUX,origin_SWF,origin_SWO,origin_SYR,origin_TLH,origin_TOL,origin_TPA,origin_TRI,origin_TTN,origin_TUL,origin_TUS,origin_TVC,origin_TXK,origin_TYR,origin_TYS,origin_UIN,origin_VLD,origin_VPS,origin_WRG,origin_XNA,origin_XWA,origin_YAK,origin_YKM,dest_ABE,dest_ABI,dest_ABQ,dest_ABR,dest_ACK,dest_ACT,dest_ACV,dest_ACY,dest_ADQ,dest_AEX,dest_AGS,dest_ALB,dest_ALO,dest_ALW,dest_AMA,dest_ANC,dest_APN,dest_ASE,dest_ATL,dest_ATW,dest_AUS,dest_AVL,dest_AVP,dest_AZA,dest_AZO,dest_BDL,dest_BFL,dest_BFM,dest_BGM

In [13]:
x_train,x_test,y_train,y_test = train_test_split(flight_model,y_target,test_size=0.3)

In [14]:
x_train_scaled = scaler.fit_transform(x_train[num_cols])
x_test_scaled = scaler.fit_transform(x_test[num_cols])
x_train[num_cols] = x_train_scaled
x_test[num_cols] = x_test_scaled

In [15]:
regressor = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, max_depth=100))
regressor.fit(x_train,y_train)
y_pred = regressor.predict(x_test)

In [16]:
y_pred

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [17]:
kfold = KFold(n_splits=5,shuffle=True)
scores = cross_validate(regressor, x_train, y_train, cv=kfold, scoring=['accuracy'])

In [18]:
print("Accuracy percentage: ",np.mean(scores['test_accuracy'])*100)

Accuracy percentage:  24.39227565316168
